# Streaming con Apache Kafka: Fundamentos

## Objetivos de Aprendizaje
- Comprender arquitectura de streaming con Apache Kafka
- Configurar productores y consumidores de Kafka
- Procesar streams de datos en tiempo real
- Implementar patrones de procesamiento de eventos
- Integrar Kafka con Python y Pandas

## Requisitos
- Python 3.8+
- kafka-python
- pandas
- Docker (para ejecutar Kafka localmente)

In [1]:
# Instalación de dependencias
import sys
!{sys.executable} -m pip install kafka-python pandas numpy -q


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Luis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import json
import time
from datetime import datetime
import pandas as pd
import numpy as np
from kafka import KafkaProducer, KafkaConsumer
from kafka.errors import KafkaError
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("Librerías cargadas correctamente")

Librerías cargadas correctamente


### 📚 **Apache Kafka: Arquitectura Event-Driven**

**Definición:**  
Apache Kafka es una plataforma distribuida de streaming de eventos que permite publicar, almacenar y procesar flujos de datos en tiempo real con alta disponibilidad y escalabilidad horizontal.

**Componentes Core:**
- **Broker**: Servidor que almacena y replica mensajes (logs inmutables)
- **Topic**: Canal lógico con categorías de mensajes (ej: `user-events`, `transactions`)
- **Partition**: División física de un topic para paralelización (0...N)
- **Producer**: Cliente que publica mensajes a topics
- **Consumer**: Cliente que lee mensajes desde topics
- **Consumer Group**: Conjunto de consumers que procesan un topic en paralelo (cada partición → 1 consumer)

**Características Críticas:**
```
📊 Throughput: Millones de mensajes/segundo
💾 Durabilidad: Persistencia en disco con replicación
🔄 Ordenamiento: Garantizado por partición (no global)
⚡ Latencia: <10ms en configuraciones optimizadas
```

**Diferencia vs Message Queues tradicionales:**
- RabbitMQ/SQS: Eliminan mensaje tras consumo (fire-and-forget)
- Kafka: Retiene mensajes según retention policy (permite replay)

**Caso de Uso:**  
Sistema de e-commerce donde cada evento (view, cart, purchase) se publica en Kafka. Múltiples consumers procesan el mismo stream: uno para analytics en tiempo real, otro para recomendaciones ML, otro para inventario.

---
**Autor:** Luis J. Raigoso V. (LJRV)

## 1. Conceptos Fundamentales de Kafka

### Arquitectura:
- **Topics**: Categorías de mensajes
- **Producers**: Publican mensajes en topics
- **Consumers**: Leen mensajes de topics
- **Brokers**: Servidores que almacenan mensajes
- **Partitions**: División de topics para escalabilidad
- **Consumer Groups**: Grupo de consumidores que procesan mensajes en paralelo

### Comandos Docker para Kafka:

```bash
# docker-compose.yml
version: '3'
services:
  zookeeper:
    image: confluentinc/cp-zookeeper:latest
    environment:
      ZOOKEEPER_CLIENT_PORT: 2181
    ports:
      - "2181:2181"

  kafka:
    image: confluentinc/cp-kafka:latest
    depends_on:
      - zookeeper
    ports:
      - "9092:9092"
    environment:
      KAFKA_BROKER_ID: 1
      KAFKA_ZOOKEEPER_CONNECT: zookeeper:2181
      KAFKA_ADVERTISED_LISTENERS: PLAINTEXT://localhost:9092
      KAFKA_OFFSETS_TOPIC_REPLICATION_FACTOR: 1
```

```bash
# Iniciar Kafka
docker-compose up -d
```

## 2. Configuración de Producer (Productor)

### 📤 **Kafka Producer: Publicación de Eventos**

**Configuraciones Críticas del Producer:**

1. **`acks` (Acknowledgment):**
   - `acks=0`: No espera confirmación (máximo throughput, sin durabilidad)
   - `acks=1`: Espera confirmación del líder (balance)
   - `acks='all'`: Espera confirmación de todos los replicas (máxima durabilidad)

2. **`retries` y `max.in.flight.requests.per.connection`:**
   ```python
   retries=3  # Reintentos automáticos ante fallos
   max_in_flight_requests_per_connection=1  # Garantiza orden con retries
   ```
   - Si `max_in_flight > 1` con retries → puede alterar orden de mensajes

3. **Serialización:**
   - JSON: Legible pero mayor tamaño
   - Avro/Protobuf: Compacto y con schema registry (recomendado producción)

**Flujo de Envío:**
```
Producer → [Partitioner] → Buffer interno → Batch → Broker → [Replication] → ACK
```

**Particionamiento:**
- Con `key`: hash(key) % num_partitions (mensajes con misma key → misma partición)
- Sin `key`: round-robin entre particiones

**Uso en el Código:**
- `send_event()`: Envío asíncrono que retorna Future
- `send_batch()`: Optimización para múltiples eventos (reduce RTT)
- `flush()`: Forzar envío de buffer antes de cerrar

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [3]:
class KafkaEventProducer:
    """Productor de eventos para Kafka"""
    
    def __init__(self, bootstrap_servers=['localhost:9092']):
        """
        Inicializar productor
        
        Args:
            bootstrap_servers: Lista de brokers de Kafka
        """
        try:
            self.producer = KafkaProducer(
                bootstrap_servers=bootstrap_servers,
                value_serializer=lambda v: json.dumps(v).encode('utf-8'),
                key_serializer=lambda k: k.encode('utf-8') if k else None,
                acks='all',  # Esperar confirmación de todos los brokers
                retries=3,
                max_in_flight_requests_per_connection=1
            )
            logger.info("Producer inicializado correctamente")
        except KafkaError as e:
            logger.error(f"Error al inicializar producer: {e}")
            self.producer = None
    
    def send_event(self, topic, key, value):
        """
        Enviar evento a Kafka
        
        Args:
            topic: Nombre del topic
            key: Clave del mensaje
            value: Valor del mensaje (dict)
        """
        if not self.producer:
            logger.error("Producer no está inicializado")
            return False
        
        try:
            future = self.producer.send(topic, key=key, value=value)
            # Esperar confirmación
            record_metadata = future.get(timeout=10)
            
            logger.info(
                f"Mensaje enviado - Topic: {record_metadata.topic}, "
                f"Partition: {record_metadata.partition}, "
                f"Offset: {record_metadata.offset}"
            )
            return True
            
        except Exception as e:
            logger.error(f"Error al enviar mensaje: {e}")
            return False
    
    def send_batch(self, topic, events):
        """
        Enviar lote de eventos
        
        Args:
            topic: Nombre del topic
            events: Lista de tuplas (key, value)
        """
        success_count = 0
        
        for key, value in events:
            if self.send_event(topic, key, value):
                success_count += 1
        
        logger.info(f"Enviados {success_count}/{len(events)} eventos")
        return success_count
    
    def close(self):
        """Cerrar producer"""
        if self.producer:
            self.producer.flush()
            self.producer.close()
            logger.info("Producer cerrado")


# Ejemplo de uso (comentado - requiere Kafka corriendo)
print("Clase KafkaEventProducer definida")
print("\nPara usar:")
print("producer = KafkaEventProducer()")
print("producer.send_event('my-topic', 'key1', {'data': 'value'})")

Clase KafkaEventProducer definida

Para usar:
producer = KafkaEventProducer()
producer.send_event('my-topic', 'key1', {'data': 'value'})


## 3. Configuración de Consumer (Consumidor)

### 📥 **Kafka Consumer: Consumo de Eventos**

**Configuraciones Esenciales:**

1. **`group_id` (Consumer Group):**
   - Consumers con mismo `group_id` → distribuyen particiones entre ellos
   - Cada partición → asignada a un solo consumer del grupo
   - Ejemplo: 3 partitions + 3 consumers → 1 partition/consumer (ideal)
   - Ejemplo: 3 partitions + 6 consumers → 3 ociosos (over-provisioning)

2. **`auto_offset_reset`:**
   - `earliest`: Leer desde el primer mensaje disponible (útil para reprocesamiento)
   - `latest`: Leer solo mensajes nuevos (default para streaming en vivo)
   - `none`: Error si no existe offset previo

3. **Commit de Offsets:**
   - `enable_auto_commit=True`: Auto-commit cada `auto_commit_interval_ms` (más simple)
   - `enable_auto_commit=False`: Manual commit tras procesamiento exitoso (más seguro)
   ```python
   consumer.commit()  # Commit manual explícito
   ```

**Gestión de Offsets:**
```
Topic: user-events
Partition 0: [msg0, msg1, msg2, msg3, msg4] → offset actual: 2 (leyó hasta msg2)
                                           ↑
                                      consumer commit
```

**Patrones de Consumo:**
- **At-most-once**: Auto-commit antes de procesar (puede perder datos)
- **At-least-once**: Commit después de procesar (puede duplicar)
- **Exactly-once**: Transacciones Kafka + idempotencia (requiere Kafka 0.11+)

**Rebalancing:**
Cuando un consumer se une/sale del grupo → Kafka redistribuye particiones (puede causar latencia temporal)

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [4]:
class KafkaEventConsumer:
    """Consumidor de eventos de Kafka"""
    
    def __init__(
        self,
        topics,
        group_id='default-group',
        bootstrap_servers=['localhost:9092']
    ):
        """
        Inicializar consumidor
        
        Args:
            topics: Lista de topics a consumir
            group_id: ID del consumer group
            bootstrap_servers: Lista de brokers
        """
        try:
            self.consumer = KafkaConsumer(
                *topics,
                bootstrap_servers=bootstrap_servers,
                group_id=group_id,
                value_deserializer=lambda m: json.loads(m.decode('utf-8')),
                key_deserializer=lambda k: k.decode('utf-8') if k else None,
                auto_offset_reset='earliest',  # Leer desde el inicio
                enable_auto_commit=True,
                auto_commit_interval_ms=1000
            )
            logger.info(f"Consumer inicializado para topics: {topics}")
        except KafkaError as e:
            logger.error(f"Error al inicializar consumer: {e}")
            self.consumer = None
    
    def consume_messages(self, max_messages=10, timeout_ms=5000):
        """
        Consumir mensajes
        
        Args:
            max_messages: Máximo de mensajes a consumir
            timeout_ms: Timeout en milisegundos
            
        Returns:
            Lista de mensajes consumidos
        """
        if not self.consumer:
            logger.error("Consumer no está inicializado")
            return []
        
        messages = []
        
        try:
            for message in self.consumer:
                msg_data = {
                    'topic': message.topic,
                    'partition': message.partition,
                    'offset': message.offset,
                    'key': message.key,
                    'value': message.value,
                    'timestamp': message.timestamp
                }
                
                messages.append(msg_data)
                logger.info(f"Mensaje consumido: {msg_data}")
                
                if len(messages) >= max_messages:
                    break
            
            return messages
            
        except Exception as e:
            logger.error(f"Error al consumir mensajes: {e}")
            return messages
    
    def consume_and_process(self, process_func, batch_size=100):
        """
        Consumir y procesar mensajes en batch
        
        Args:
            process_func: Función para procesar batch
            batch_size: Tamaño del batch
        """
        if not self.consumer:
            return
        
        batch = []
        
        try:
            for message in self.consumer:
                batch.append(message.value)
                
                if len(batch) >= batch_size:
                    # Procesar batch
                    process_func(batch)
                    batch = []
                    
        except KeyboardInterrupt:
            logger.info("Consumo interrumpido por usuario")
            if batch:
                process_func(batch)
    
    def close(self):
        """Cerrar consumer"""
        if self.consumer:
            self.consumer.close()
            logger.info("Consumer cerrado")


print("Clase KafkaEventConsumer definida")

Clase KafkaEventConsumer definida


## 4. Simulación de Streaming (Sin Kafka Real)

### 🎭 **Simulación de Eventos: Testing Sin Infraestructura**

**¿Por qué Simular?**
1. Kafka requiere infraestructura (Zookeeper + Broker) que puede no estar disponible en notebooks
2. Permite desarrollar lógica de procesamiento sin dependencias externas
3. Útil para testing unitario y desarrollo offline

**Patrón Event Sourcing:**
Los eventos capturan *cambios de estado* en vez de estado actual:
```
Estado actual:        cart = {items: 3, total: 150}
Event Sourcing:       [ProductAdded, ProductAdded, ProductAdded, CouponApplied]
```

**Beneficios:**
- Auditoría completa (replay de eventos)
- Debugging: Reproducir bugs desde secuencia de eventos
- Múltiples vistas (projections) del mismo stream

**Características del Simulador:**
- `generate_event()`: Crea eventos realistas con distribuciones aleatorias
- `generate_stream()`: Simula flujo continuo con delays configurables
- Delay pequeño (0.01s) para testing rápido, mayor (1s) para simular real-time

**Eventos de E-commerce:**
- `view`: Usuario visualiza producto (top funnel)
- `add_to_cart`: Agrega al carrito (middle funnel)
- `purchase`: Completa transacción (conversion)
- `remove_from_cart`: Abandono (negative signal)

**Aplicación Real:**  
Este simulador representa un sistema de tracking de eventos similar a Google Analytics, Segment, o Amplitude.

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [5]:
# Simulador de eventos de e-commerce
class EcommerceEventSimulator:
    """Simulador de eventos de e-commerce en tiempo real"""
    
    def __init__(self):
        self.event_types = ['view', 'add_to_cart', 'purchase', 'remove_from_cart']
        self.products = ['laptop', 'mouse', 'keyboard', 'monitor', 'headphones']
        self.users = [f'user_{i}' for i in range(1, 101)]
    
    def generate_event(self):
        """Generar un evento aleatorio"""
        return {
            'event_id': np.random.randint(1000000, 9999999),
            'timestamp': datetime.now().isoformat(),
            'user_id': np.random.choice(self.users),
            'event_type': np.random.choice(self.event_types),
            'product': np.random.choice(self.products),
            'price': round(np.random.uniform(10, 2000), 2),
            'quantity': np.random.randint(1, 5)
        }
    
    def generate_stream(self, n_events=100, delay=0.1):
        """Generar stream de eventos"""
        events = []
        
        for _ in range(n_events):
            event = self.generate_event()
            events.append(event)
            print(f"Evento generado: {event['event_type']} - {event['product']}")
            time.sleep(delay)
        
        return events


# Generar eventos de ejemplo
simulator = EcommerceEventSimulator()
print("\nGenerando 10 eventos de ejemplo...\n")
sample_events = simulator.generate_stream(n_events=10, delay=0.01)

# Convertir a DataFrame
df_events = pd.DataFrame(sample_events)
print("\nDataFrame de eventos:")
df_events


Generando 10 eventos de ejemplo...

Evento generado: add_to_cart - headphones
Evento generado: view - headphones
Evento generado: purchase - keyboard
Evento generado: view - mouse
Evento generado: view - mouse
Evento generado: purchase - mouse
Evento generado: add_to_cart - keyboard
Evento generado: add_to_cart - mouse
Evento generado: purchase - headphones
Evento generado: remove_from_cart - headphones

DataFrame de eventos:


,event_id,timestamp,user_id,event_type,product,price,quantity
0,4197647,2025-12-07T17:53:33.659625,user_16,add_to_cart,headphones,1071.66,3
1,8042572,2025-12-07T17:53:33.670724,user_98,view,headphones,1882.54,1
2,4487051,2025-12-07T17:53:33.681593,user_22,purchase,keyboard,90.49,4
3,1624807,2025-12-07T17:53:33.692236,user_95,view,mouse,140.07,2
4,5479196,2025-12-07T17:53:33.702997,user_67,view,mouse,21.96,2
5,8140584,2025-12-07T17:53:33.713322,user_26,purchase,mouse,210.78,4
6,8334735,2025-12-07T17:53:33.724148,user_39,add_to_cart,keyboard,1416.38,1
7,6415800,2025-12-07T17:53:33.734939,user_57,add_to_cart,mouse,1231.09,2
8,2513134,2025-12-07T17:53:33.745345,user_43,purchase,headphones,1330.04,3
9,4309893,2025-12-07T17:53:33.756077,user_95,remove_from_cart,headphones,1525.67,4


## 5. Procesamiento de Stream en Tiempo Real

### ⚙️ **Stream Processing: Análisis en Tiempo Real**

**Window-Based Processing:**

1. **Tumbling Window (Ventana Fija):**
   ```
   [0...10] [10...20] [20...30] ← No solapamiento
   ```
   - Cada evento pertenece a una única ventana
   - Útil para métricas periódicas: "ventas por minuto"

2. **Sliding Window (Ventana Deslizante):**
   ```
   [0...10]
       [5...15]
           [10...20] ← Solapamiento
   ```
   - Ventanas con overlap (implementada en código)
   - Útil para tendencias suaves: "promedio móvil últimos 5 min"

3. **Session Window:**
   - Agrupa eventos por inactividad
   - Ejemplo: "sesión de usuario termina tras 30 min sin actividad"

**Métricas de Streaming:**
- **Throughput**: Eventos procesados por segundo
- **Latency**: Tiempo desde evento → procesamiento
- **Watermarks**: Manejo de eventos fuera de orden (late arrivals)

**Estado del Procesador:**
```python
self.stats = {
    'total_events': 0,        # Contador acumulativo
    'events_by_type': {},     # Agregación por dimensión
    'total_revenue': 0        # Métrica de negocio crítica
}
```

**Procesamiento Stateful vs Stateless:**
- **Stateless**: Cada evento se procesa independientemente (map, filter)
- **Stateful**: Mantiene estado entre eventos (aggregations, joins) ← Este código

**Aplicación Real:**  
Similar a Apache Flink/Spark Streaming para dashboards en vivo mostrando KPIs actualizados continuamente.

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [6]:
class StreamProcessor:
    """Procesador de streams de datos"""
    
    def __init__(self, window_size=10):
        self.window_size = window_size
        self.events_buffer = []
        self.stats = {
            'total_events': 0,
            'events_by_type': {},
            'total_revenue': 0
        }
    
    def process_event(self, event):
        """Procesar un evento individual"""
        self.events_buffer.append(event)
        self.stats['total_events'] += 1
        
        # Contar por tipo
        event_type = event['event_type']
        self.stats['events_by_type'][event_type] = \
            self.stats['events_by_type'].get(event_type, 0) + 1
        
        # Calcular revenue para purchases
        if event_type == 'purchase':
            self.stats['total_revenue'] += event['price'] * event['quantity']
        
        # Procesar ventana si está llena
        if len(self.events_buffer) >= self.window_size:
            self.process_window()
    
    def process_window(self):
        """Procesar ventana de eventos"""
        df_window = pd.DataFrame(self.events_buffer)
        
        print("\n" + "="*50)
        print(f"PROCESANDO VENTANA DE {len(df_window)} EVENTOS")
        print("="*50)
        
        # Análisis de la ventana
        print("\nEventos por tipo:")
        print(df_window['event_type'].value_counts())
        
        print("\nProductos más populares:")
        print(df_window['product'].value_counts().head())
        
        # Tasa de conversión
        views = len(df_window[df_window['event_type'] == 'view'])
        purchases = len(df_window[df_window['event_type'] == 'purchase'])
        conversion_rate = (purchases / views * 100) if views > 0 else 0
        
        print(f"\nTasa de conversión: {conversion_rate:.2f}%")
        
        # Revenue de la ventana
        window_revenue = df_window[df_window['event_type'] == 'purchase'].apply(
            lambda row: row['price'] * row['quantity'], axis=1
        ).sum()
        
        print(f"Revenue de la ventana: ${window_revenue:,.2f}")
        
        # Limpiar buffer
        self.events_buffer = []
    
    def get_stats(self):
        """Obtener estadísticas globales"""
        return self.stats


# Procesar eventos simulados
processor = StreamProcessor(window_size=20)

print("Generando y procesando 50 eventos...")
events = simulator.generate_stream(n_events=50, delay=0.01)

for event in events:
    processor.process_event(event)

# Mostrar estadísticas finales
print("\n" + "="*50)
print("ESTADÍSTICAS FINALES")
print("="*50)
stats = processor.get_stats()
print(f"\nTotal de eventos: {stats['total_events']}")
print(f"\nEventos por tipo:")
for event_type, count in stats['events_by_type'].items():
    print(f"  {event_type}: {count}")
print(f"\nRevenue total: ${stats['total_revenue']:,.2f}")

Generando y procesando 50 eventos...
Evento generado: view - keyboard
Evento generado: remove_from_cart - keyboard
Evento generado: purchase - keyboard
Evento generado: purchase - keyboard
Evento generado: purchase - laptop
Evento generado: remove_from_cart - laptop
Evento generado: purchase - keyboard
Evento generado: add_to_cart - headphones
Evento generado: purchase - laptop
Evento generado: remove_from_cart - laptop
Evento generado: add_to_cart - laptop
Evento generado: remove_from_cart - laptop
Evento generado: view - laptop
Evento generado: view - keyboard
Evento generado: purchase - mouse
Evento generado: purchase - keyboard
Evento generado: add_to_cart - keyboard
Evento generado: remove_from_cart - keyboard
Evento generado: view - keyboard
Evento generado: remove_from_cart - monitor
Evento generado: remove_from_cart - headphones
Evento generado: purchase - keyboard
Evento generado: remove_from_cart - keyboard
Evento generado: add_to_cart - monitor
Evento generado: view - monito

## 6. Patrones de Procesamiento de Streams

### 🔄 **Patrones Avanzados: Sliding Windows y Agregaciones**

**Implementación de Sliding Window:**
```python
window_size = 10  # Tamaño de ventana
slide = 5         # Paso de deslizamiento
```
- `slide < window_size`: Ventanas solapadas (más suave, más cómputo)
- `slide = window_size`: Equivalente a tumbling window
- `slide > window_size`: Gap entre ventanas (puede perder eventos)

**Ventana Deslizante en Acción:**
```
Eventos:  [e0, e1, e2, e3, e4, e5, e6, e7, e8, e9, e10...]
Window 1: [e0..................e9]
Window 2:          [e5..................e14]
Window 3:                   [e10..................e19]
```

**Métricas Calculadas:**
1. **Total Events**: Conteo simple por ventana
2. **Unique Users**: Cardinalidad (distinct count) para medir alcance
3. **Total Revenue**: Suma condicional solo de eventos `purchase`

**Optimización para Producción:**
- **Incremental Aggregation**: No recalcular todo, solo añadir/remover eventos del borde
- **Late Data Handling**: Watermarks para eventos que llegan con delay
- **State Backends**: Almacenar estado en RocksDB para ventanas grandes

**Comparación con Batch:**
```
Batch:     [esperar 1 hora] → procesar todos → resultado
Streaming: evento → [ventana] → resultado parcial cada 5 min
```

**Casos de Uso:**
- Detección de anomalías: Spike en errores en ventana de 5 min
- Trending topics: Productos más vistos en últimos 10 min
- Conversion rate: Views vs purchases en ventana deslizante

**Limitaciones de la Implementación Actual:**
- Usa memoria (en producción → state backend distribuido)
- No maneja eventos out-of-order (en producción → event-time processing)

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [7]:
# Patrón 1: Agregaciones por ventana deslizante
def sliding_window_aggregation(events, window_size=10, slide=5):
    """
    Agregaciones con ventana deslizante
    
    Args:
        events: Lista de eventos
        window_size: Tamaño de la ventana
        slide: Cuántos eventos deslizar
    """
    results = []
    
    for i in range(0, len(events) - window_size + 1, slide):
        window = events[i:i + window_size]
        df_window = pd.DataFrame(window)
        
        agg_result = {
            'window_start': i,
            'window_end': i + window_size,
            'total_events': len(df_window),
            'unique_users': df_window['user_id'].nunique(),
            'total_revenue': df_window[df_window['event_type'] == 'purchase'].apply(
                lambda row: row['price'] * row['quantity'], axis=1
            ).sum()
        }
        
        results.append(agg_result)
    
    return pd.DataFrame(results)


# Aplicar ventana deslizante
events_for_window = simulator.generate_stream(n_events=100, delay=0)
df_sliding = sliding_window_aggregation(events_for_window, window_size=20, slide=10)

print("Agregaciones por ventana deslizante:")
print(df_sliding)

Evento generado: add_to_cart - headphones
Evento generado: add_to_cart - mouse
Evento generado: purchase - mouse
Evento generado: add_to_cart - monitor
Evento generado: add_to_cart - laptop
Evento generado: remove_from_cart - monitor
Evento generado: add_to_cart - headphones
Evento generado: remove_from_cart - headphones
Evento generado: remove_from_cart - laptop
Evento generado: purchase - laptop
Evento generado: view - keyboard
Evento generado: view - keyboard
Evento generado: remove_from_cart - mouse
Evento generado: add_to_cart - headphones
Evento generado: purchase - monitor
Evento generado: purchase - mouse
Evento generado: remove_from_cart - laptop
Evento generado: add_to_cart - mouse
Evento generado: purchase - laptop
Evento generado: purchase - monitor
Evento generado: purchase - laptop
Evento generado: remove_from_cart - monitor
Evento generado: add_to_cart - headphones
Evento generado: add_to_cart - mouse
Evento generado: remove_from_cart - monitor
Evento generado: add_to_ca

## Resumen y Mejores Prácticas

### Conceptos Clave de Kafka:
1. **Durabilidad**: Los mensajes se persisten en disco
2. **Escalabilidad**: Particiones para procesamiento paralelo
3. **Alto throughput**: Optimizado para grandes volúmenes
4. **Ordenamiento**: Garantizado dentro de una partición
5. **Consumer Groups**: Procesamiento distribuido

### Mejores Prácticas:
- Usar claves de mensaje para particionamiento consistente
- Configurar replicación para alta disponibilidad
- Monitorear lag de consumers
- Implementar idempotencia en consumers
- Usar compression para reducir tamaño de mensajes
- Definir retention policies apropiadas

### Patrones de Streaming:
- **Event Sourcing**: Almacenar cambios como eventos
- **CQRS**: Separar lecturas y escrituras
- **Windowing**: Procesar en ventanas de tiempo
- **Aggregations**: Sumar, contar, promediar en ventanas
- **Joins**: Combinar streams relacionados

### Casos de Uso:
- Análisis en tiempo real
- Monitoreo de sistemas
- Detección de fraude
- Recomendaciones personalizadas
- IoT y telemetría

### Recursos Adicionales:
- [Apache Kafka Documentation](https://kafka.apache.org/documentation/)
- [Kafka Python Client](https://kafka-python.readthedocs.io/)
- [Stream Processing Patterns](https://www.confluent.io/blog/streaming-data-patterns/)

---

## 🧭 Navegación

**← Anterior:** [⚡ Mid - 01. Orquestación de Pipelines con Apache Airflow](01_apache_airflow_fundamentos.ipynb)

**Siguiente →:** [☁️ AWS para Ingeniería de Datos: S3, Glue, Athena y Lambda →](03_cloud_aws.ipynb)

**📚 Índice de Nivel Mid:**
- [⚡ Mid - 01. Orquestación de Pipelines con Apache Airflow](01_apache_airflow_fundamentos.ipynb)
- [Streaming con Apache Kafka: Fundamentos](02_streaming_kafka.ipynb) ← 🔵 Estás aquí
- [☁️ AWS para Ingeniería de Datos: S3, Glue, Athena y Lambda](03_cloud_aws.ipynb)
- [☁️ GCP para Ingeniería de Datos: BigQuery, Cloud Storage, Dataflow y Composer](03b_cloud_gcp.ipynb)
- [☁️ Azure para Ingeniería de Datos: ADLS, Synapse, Data Factory y Databricks](03c_cloud_azure.ipynb)
- [🗄️ Bases de Datos Relacionales y NoSQL: PostgreSQL y MongoDB](04_bases_datos_postgresql_mongodb.ipynb)
- [♻️ DataOps y CI/CD para Pipelines de Datos](05_dataops_cicd.ipynb)
- [🌐 Conectores Avanzados: REST, GraphQL y SFTP](06_conectores_avanzados_rest_graphql_sftp.ipynb)
- [🧩 Optimización SQL y Particionado de Datos](07_optimizacion_sql_particionado.ipynb)
- [🚀 Servicios de Datos con FastAPI](08_fastapi_servicios_datos.ipynb)
- [🧪 Proyecto Integrador Mid 1: API → DB → Parquet con Orquestación](09_proyecto_integrador_1.ipynb)
- [🔄 Proyecto Integrador Mid 2: Kafka → Streaming → Data Lake y Monitoreo](10_proyecto_integrador_2.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
